In [1]:
!pip install kagglehub
# Import required libraries
import kagglehub
import os
import shutil # New library for safe file copying
from glob import glob
import torch
import torchaudio as ta
from chatterbox.tts import ChatterboxTTS
from IPython.display import Audio, display

C:\Users\Karim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\perth\perth_net\__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [16]:
import kagglehub
import os
from glob import glob

# --- TESS DATASET SETUP ---
# 1. Download the TESS dataset
data_path = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")

# The actual audio files are nested inside the two actor folders
# Use glob to find all .wav files in the downloaded structure
all_files = glob(os.path.join(data_path, '**', '*.wav'), recursive=True)

# 2. Define target emotions and actor
# TESS only has two female actors: 'YAF' and 'OAF'. We'll use the first one, 'YAF'.
TARGET_ACTOR_PREFIX = 'YAF'

# Define the target emotions based on the folder/filename structure:
# 'happy', 'sad', 'angry'
TARGET_EMOTIONS = ['happy', 'sad', 'angry']

print(f"Total files found in TESS dataset: {len(all_files)}")

# 3. Create a dictionary to store the reference clip found for each emotion
reference_clips = {}

# 4. Loop through all files and apply the filters
for file_path in all_files:
    filename = os.path.basename(file_path)

    # TESS filenames are typically in the format: ActorEmotion_word_filename.wav
    # e.g., 'YAF_happy_boat.wav' or 'OAF_sad_pass.wav'

    # Filter 1: Check if the file is from the target actor (YAF)
    if not filename.startswith(TARGET_ACTOR_PREFIX):
        continue

    # Filter 2: Check if the file is one of the target emotions
    # We check if the filename contains the emotion name (e.g., 'happy')
    for emotion_name in TARGET_EMOTIONS:
        if emotion_name in filename:

            # We only need one example per emotion. We will take the first one found.
            # This ensures we get a clip, but it will be arbitrary (e.g., 'YAF_happy_boat.wav')
            if emotion_name not in reference_clips:
                reference_clips[emotion_name.capitalize()] = file_path

# 5. Print the selected files and create a clean directory for them
output_dir = 'tess_reference_clips'
os.makedirs(output_dir, exist_ok=True)
print("\n--- Selected Reference Clips ---")

for emotion, source_path in reference_clips.items():
    target_filename = f"{emotion.lower()}_ref_actor_{TARGET_ACTOR_PREFIX}.wav"
    target_path = os.path.join(output_dir, target_filename)

    shutil.copy(source_path, target_path)

    print(f"[{emotion}]: {os.path.basename(source_path)} copied to {target_path}")
    

print(f"\nPreparation complete. {len(reference_clips)} clips ready in the '{output_dir}' directory.")

Total files found in TESS dataset: 5600

--- Selected Reference Clips ---
[Angry]: YAF_youth_angry.wav copied to tess_reference_clips\angry_ref_actor_YAF.wav
[Happy]: YAF_youth_happy.wav copied to tess_reference_clips\happy_ref_actor_YAF.wav
[Sad]: YAF_youth_sad.wav copied to tess_reference_clips\sad_ref_actor_YAF.wav

Preparation complete. 3 clips ready in the 'tess_reference_clips' directory.


In [17]:
import os
import torch
# Add necessary imports that were likely missing from your full notebook:
import torchaudio as ta 
from chatterbox import ChatterboxTTS
from IPython.display import Audio, display
# Other imports used:
import numpy as np 
from transformers import pipeline 

# --- CRITICAL FIX: Re-define variables from Part 1 ---
# This ensures the Chatterbox block uses the correct actor and output directory
output_dir = 'tess_reference_clips'
TARGET_ACTOR_PREFIX = 'YAF' 

In [18]:
print("Files inside tess_reference_clips:")
print(os.listdir(output_dir))


Files inside tess_reference_clips:
['angry_ref_actor_YAF.wav', 'happy_ref_actor_YAF.wav', 'sad_ref_actor_YAF.wav']


In [22]:
# ---
# NOTE: The installation block is run automatically once, but here for completeness:
# !pip install chatterbox-tts torch torchaudio numpy soundfile 

## --- PART 2: CHATTERBOX EXECUTION (NO CHANGES NEEDED HERE, as os.path.join is used) ---

# The text you want to generate
TARGET_TEXT = "Mert is amazing, i love karim"

# The emotion clips you prepared (will now reference the safely copied files)
REFERENCE_CLIPS = {
    'Happy': os.path.join("tess_reference_clips", "happy_ref_actor_YAF.wav"),
    'Sad': os.path.join("tess_reference_clips", "sad_ref_actor_YAF.wav"),
    'Angry': os.path.join("tess_reference_clips", "angry_ref_actor_YAF.wav"),
}


# Configure output folder
output_folder = "synthesized_chatterbox_emotions"
os.makedirs(output_folder, exist_ok=True)

# 2. LOAD THE CHATTERBOX MODEL 
if torch.cuda.is_available():
    device = "cuda"
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"
    print("Warning: Running on CPU. Generation may be very slow.")

print(f"Loading Chatterbox model on device: {device}...")
try:
    model = ChatterboxTTS.from_pretrained(device=device)
    print("Chatterbox model loaded successfully.")
except Exception as e:
    print(f"\nERROR: Failed to load Chatterbox model. Error: {e}")
    exit()

print(f"\nTarget Text: **{TARGET_TEXT}**")
print("--------------------------------------------------")

# 3. PERFORM ZERO-SHOT CLONING 
EXAGGERATION_LEVELS = {
    'Neutral': 0.7,
    'Happy': 0.7,
    'Sad': 0.7,
    'Angry': 0.7,
    'Fearful': 0.7,
}

for emotion_name, ref_path in REFERENCE_CLIPS.items():
    print(f"\n🎙️ Cloning **{emotion_name}** emotion...")

    output_wav_path = os.path.join(output_folder, f"{emotion_name.lower()}_chatterbox_cloned.wav")
    
    wav = model.generate(
        text=TARGET_TEXT,
        audio_prompt_path=ref_path,
        exaggeration=EXAGGERATION_LEVELS[emotion_name]
    )

    # Save audio
    wav_np = wav.cpu().numpy().squeeze()
    sf.write(output_wav_path, wav_np, model.sr)

    print(f"Generated file: {output_wav_path} (Exaggeration: {EXAGGERATION_LEVELS[emotion_name]})")
    display(Audio(output_wav_path, autoplay=False))


Loading Chatterbox model on device: cpu...
loaded PerthNet (Implicit) at step 250,000
Chatterbox model loaded successfully.

Target Text: **Mert is amazing, i love karim**
--------------------------------------------------

🎙️ Cloning **Happy** emotion...



Sampling:   9%|██████                                                                | 87/1000 [00:14<02:29,  6.11it/s]


Generated file: synthesized_chatterbox_emotions\happy_chatterbox_cloned.wav (Exaggeration: 0.7)



🎙️ Cloning **Sad** emotion...



Sampling:  11%|███████▋                                                             | 111/1000 [00:19<02:40,  5.55it/s]


Generated file: synthesized_chatterbox_emotions\sad_chatterbox_cloned.wav (Exaggeration: 0.7)



🎙️ Cloning **Angry** emotion...


Sampling:   7%|████▋                                                                 | 67/1000 [00:12<02:56,  5.28it/s]


Generated file: synthesized_chatterbox_emotions\angry_chatterbox_cloned.wav (Exaggeration: 0.7)


In [27]:
import whisper
import os
from jiwer import wer
import soundfile as sf
import numpy as np

# Load Whisper
model = whisper.load_model("base")

# Target text
TARGET_TEXT = "Mert is amazing, i love karim"

# Folder with generated audios
output_folder = "synthesized_chatterbox_emotions"
generated_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".wav")]

print("=== WER Evaluation (Using soundfile) ===")

for file in generated_files:
    try:
        # Load audio (already returns float64 by default)
        audio_data, sr = sf.read(file)
        
        # Convert to float32
        audio_data = audio_data.astype(np.float32)
        
        # If stereo, convert to mono
        if audio_data.ndim > 1:
            audio_data = audio_data.mean(axis=1)
        
        # Resample to 16000 if needed (optional, install resampy if you don't have it)
        if sr != 16000:
            import resampy
            audio_data = resampy.resample(audio_data, sr, 16000)
        
    except Exception as e:
        print(f"Error loading {os.path.basename(file)}: {e}")
        continue

    # Transcribe using Whisper
    result = model.transcribe(audio_data, fp16=False)  # fp16=False if CPU
    transcription = result["text"]
    
    # Compute WER
    score = wer(TARGET_TEXT.lower(), transcription.lower())
    
    print(f"{os.path.basename(file)} -> WER: {score:.3f}, Transcription: {transcription}")

print("\nEvaluation complete.")



=== WER Evaluation (Using soundfile) ===
angry_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Murt is amazing. I love Karim.
happy_chatterbox_cloned.wav -> WER: 0.500, Transcription:  MIRR is amazing. I love Kareem.
sad_chatterbox_cloned.wav -> WER: 0.500, Transcription:  Myrt is amazing. I love cream.

Evaluation complete.


In [28]:
import os
import numpy as np
import soundfile as sf
from transformers import pipeline

# Folder with your generated audios
output_folder = "synthesized_chatterbox_emotions"
generated_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".wav")]

# Expected emotions
EXPECTED_EMOTIONS = {
    'angry_chatterbox_cloned.wav': 'Angry',
    'fearful_chatterbox_cloned.wav': 'Fearful',
    'happy_chatterbox_cloned.wav': 'Happy',
    'neutral_chatterbox_cloned.wav': 'Neutral',
    'sad_chatterbox_cloned.wav': 'Sad',
}

# Load the SER pipeline
print("Loading Speech Emotion Recognition model...")
emotion_classifier = pipeline(
    "audio-classification",
    model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition",
    device=-1  # CPU
)
TARGET_SR = emotion_classifier.feature_extractor.sampling_rate
print(f"SER model loaded successfully. Target Sample Rate: {TARGET_SR} Hz\n")

# Function to resample audio if needed
def resample_audio(audio, orig_sr, target_sr):
    if orig_sr != target_sr:
        import resampy
        audio = resampy.resample(audio, orig_sr, target_sr)
    return audio

# Evaluate each audio file
print("=== Emotion Recognition Results ===")
print("{:<25} {:<10} {:<10} {:<15}".format("Audio File", "Target", "Predicted", "Confidence"))
print("-" * 60)

for file in generated_files:
    filename = os.path.basename(file)
    target_emotion = EXPECTED_EMOTIONS.get(filename, "N/A")
    
    try:
        # Load audio with soundfile (avoids torchaudio/TorchCodec)
        audio_data, sr = sf.read(file)
        
        # Convert to float32
        audio_data = audio_data.astype(np.float32)
        
        # If stereo, convert to mono
        if audio_data.ndim > 1:
            audio_data = audio_data.mean(axis=1)
        
        # Resample if necessary
        audio_data = resample_audio(audio_data, sr, TARGET_SR)
        
    except Exception as e:
        print(f"Error loading {filename}: {e}")
        continue
    
    # Predict emotion
    prediction = emotion_classifier(audio_data)
    
    if prediction:
        predicted_emotion = prediction[0]['label'].capitalize()
        confidence = prediction[0]['score']
        match_status = "✅ Match" if predicted_emotion == target_emotion else "❌ Mismatch"
        
        print("{:<25} {:<10} {:<10} {:<15.3f} ({})".format(
            filename,
            target_emotion,
            predicted_emotion,
            confidence,
            match_status
        ))

print("\nEvaluation complete. This method bypasses FFmpeg/TorchCodec completely.")


Loading Speech Emotion Recognition model...


C:\Users\Karim\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\configuration_utils.py:306: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining mo

SER model loaded successfully. Target Sample Rate: 16000 Hz

=== Emotion Recognition Results ===
Audio File                Target     Predicted  Confidence     
------------------------------------------------------------
angry_chatterbox_cloned.wav Angry      Calm       0.132           (❌ Mismatch)
happy_chatterbox_cloned.wav Happy      Calm       0.134           (❌ Mismatch)
sad_chatterbox_cloned.wav Sad        Calm       0.137           (❌ Mismatch)

Evaluation complete. This method bypasses FFmpeg/TorchCodec completely.
